In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import sys
sys.path.append('C:\\Users\\user\\PycharmProjects\\cobrave')
from cobrave.plotting import plot_clustermap
import cobrave
from cobrave.plotting import plot_embedding, plot_PCA

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LassoLars
from sklearn.manifold import TSNE, Isomap, MDS, SpectralEmbedding, LocallyLinearEmbedding
import umap

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import NearestCentroid
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, mutual_info_classif
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

import lightgbm as lgb

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [3]:
def transform_func(x):
    return np.log2(x + 1)

def training(X, y, clf_type=LogisticRegression, cv=5, **kwargs):
    clf = make_pipeline(
        #StandardScaler(), VarianceThreshold(), 
        #SelectKBest(f_classif, k=3000), 
        #SelectKBest(mutual_info_classif, k=3000), 
                        clf_type(**kwargs))

    scores = cross_val_score(clf, X, y, cv=cv, verbose=1)
        
    sns.histplot(scores)
    print(sum(scores) / len(scores))
    return scores


def GS_training(X, y, parameters, clf_type=LogisticRegression, cv=5, **kwargs):
    clf = GridSearchCV(clf_type(**kwargs), parameters, cv=cv, verbose=2, scoring='f1_micro')
    result = clf.fit(X, y)
    return result

In [46]:
fpath = Path("../results/2022March/ML/data/RLE.tsv")
fpath_2 = Path("../results/2022March/ML/data/rxn_score.tsv")
# fpath_3 = Path("../results/2022March/ML/data/task_score.tsv")
fpath_4 = Path("../results/2022March/ML/data/pfba.csv")
outliers = [] # ["R031A","R040A","R050A", "R046A", "R054A", "R053A"]

In [47]:
df = transform_func(pd.read_csv(fpath, sep='\t', index_col=0))
df2 = pd.read_csv(fpath_2, sep='\t', index_col=0)
df3 = pd.read_csv(fpath_3, sep='\t', index_col=0)
df4 = pd.read_csv(fpath_4, index_col=0)

In [104]:
df2.index = df2.index.to_series().apply(lambda x: "rs_" + x)

In [106]:
df4.index = df4.index.to_series().apply(lambda x: "fx_" + x)

## concat data

In [107]:
mg_df = pd.concat([df, df2,df3, df4], axis=0)
mg_df = mg_df.loc[:, [c for c in mg_df.columns if c not in outliers]]

In [8]:
labels = pd.read_excel("../data/all_info/labels.xlsx", sheet_name="20220203")
gb = labels.groupby("group")["sample"].apply(lambda x: list(x))
groups = {f"G{i+1}": [g + "A" for g in gs] for i, gs in enumerate(gb)}
groups["G3"].append("R016A.1")
groups = {k: [vi for vi in v if vi not in outliers] for k, v in groups.items()}

In [108]:
tv_df, test_df = mg_df.loc[:, groups["G1"]+groups["G2"]+groups["G4"]], mg_df.loc[:, groups["G3"]]

## make features + labels for training

In [11]:
y = []
for c in tv_df.columns:
    g = "G1" if c in groups["G1"] else "G2" if c in groups["G2"] else "G4" if c in groups["G4"] else ""
    assert g != ""
    y.append(g)

In [109]:
X, y = tv_df.values.T, np.array(y)

In [110]:
clf = make_pipeline(StandardScaler(), VarianceThreshold(), 
                    SelectKBest(f_classif, k=4000)
                   )

nX = clf.fit_transform(X, y)

In [226]:
test_df

,R004A,R006A,R016A,R027A,R029A,R030A,R045A,R016A.1
ENSG00000000003.13,3.123990,3.226544,1.436905,3.114609,4.138414,1.434475,0.000000,3.074362
ENSG00000000419.11,8.073325,8.130476,8.479747,8.947290,8.555821,8.788016,7.534856,8.887550
ENSG00000000938.11,13.057616,11.606263,12.910896,12.322467,11.013366,13.114201,12.902943,12.592707
ENSG00000001036.12,8.288574,7.659660,8.349873,8.144359,7.761181,9.229777,7.979430,8.206781
ENSG00000001084.9,10.294006,11.110773,10.832609,9.986709,9.935547,11.783068,11.633827,11.006478
...,...,...,...,...,...,...,...,...
fx_MAR08663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
fx_MAR08667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
fx_MAR08668,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
fx_MAR10393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [207]:
test_X =clf.transform(test_df.fillna(0).values.T)

In [114]:
features = tv_df.index

In [72]:
clf

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('variancethreshold', VarianceThreshold()),
                ('selectkbest', SelectKBest(k=10000))])

In [115]:
final_ftr = features[clf["variancethreshold"].get_support()][clf["selectkbest"].get_support()]

In [116]:
final_ftr

Index(['ENSG00000000938.11', 'ENSG00000002330.12', 'ENSG00000002834.16',
       'ENSG00000002919.13', 'ENSG00000003056.6', 'ENSG00000003402.18',
       'ENSG00000003756.15', 'ENSG00000004059.9', 'ENSG00000004399.11',
       'ENSG00000004779.8',
       ...
       'fx_MAR11066', 'fx_MAR11174', 'fx_MAR11205', 'fx_MAR11316',
       'fx_MAR11506', 'fx_MAR11507', 'fx_MAR11508', 'fx_MAR11953',
       'fx_MAR11894', 'fx_MAR11895'],
      dtype='object', length=4000)

In [123]:
len(final_ftr[final_ftr.to_series().apply(lambda x: "ENSG" in x)])

3408

In [122]:
len(final_ftr[final_ftr.to_series().apply(lambda x: "General" in x)])

21

In [121]:
len(final_ftr[final_ftr.to_series().apply(lambda x: "rs" in x)])

491

In [120]:
len(final_ftr[final_ftr.to_series().apply(lambda x: "fx" in x)])

80

## Training ML models

In [ ]:
params = {'C': [0.1, 1, 10, 100], 
          #"l1_ratio": np.arange(0.1, 1, 0.2),
          "class_weight": [None, "balanced"]}
logic_scores = GS_training(nX, y, params, LogisticRegression, cv=7,
                           random_state=0, penalty="l2", max_iter=2000)

In [230]:
logic_scores.best_score_

0.7959183673469388

In [231]:
logic_es = logic_scores.best_estimator_.fit(nX, y)
logic_es.predict(test_X)

array(['G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4'], dtype='<U2')

In [ ]:
params = {'C': [0.01, 0.1, 1, 10, 50, 100, 1000], 
          'degree': np.arange(1,5,1), 
          'kernel': ('poly', 'rbf'), "gamma": [1e-2, 0.001, 0.005, 0.0001]}

svm_scores = GS_training(nX, y, params, SVC, gamma="auto", cv=7)

In [234]:
svm_scores.best_score_ # 0.7933333333333333

0.7755102040816325

In [235]:
svm_es = svm_scores.best_estimator_.fit(nX, y)
svm_es.predict(test_X)

array(['G1', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4'], dtype='<U2')

In [ ]:
params = {"learning_rate": [0.01, 0.03, 0.1, 0.3],
          "n_estimators": [100, 1000, 2000],
          "subsample": [1],
          "min_samples_split": [2, 5, 10, 15, 20],
          "max_features": ("sqrt", "log2"),
          "min_samples_leaf": [8], 
         }

params = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

gbm_scores = GS_training(nX, y, params, GradientBoostingClassifier)

In [188]:
gbm_scores.best_score_

0.8377777777777776

In [189]:
gbm_scores.best_estimator_

GradientBoostingClassifier(learning_rate=0.2, max_depth=5, max_features='sqrt',
                           min_samples_leaf=0.31818181818181823,
                           min_samples_split=0.24545454545454548,
                           n_estimators=10)

In [236]:
gbm_es = gbm_scores.best_estimator_.fit(nX, y)
gbm_es.predict(test_X)

array(['G1', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4'], dtype='<U2')

In [ ]:
params = {"num_leaves": [4, 8],
          "max_depth": [2, 4, 6, 10],
          "n_estimators": [5, 20, 50, 100],
          "class_weight": ["balanced"],
          "min_child_samples": [10],
          #"min_split_gain": [0.0,0.3,0.8,1.0],
          "colsample_bytree":  [0.4],
          "reg_alpha": np.arange(0.2, 0.6, 0.4),
          # "reg_beta": [0, 0.3, 0.6],
          "learning_rate": [0.1, 0.05, 0.01],
          #"subsample_freq": [0, 0.3],
          'feature_fraction': [0.2, 0.8, 1.0],
          'bagging_fraction': [0.2, 0.8, 1.0],
          'bagging_freq': range(0, 81, 40),
          #"subsample": [0.8, 1],
          "subsample_for_bin": [10, 500, 100]
         }
lgb_scores = GS_training(nX, y, params, lgb.LGBMClassifier)

In [255]:
lgb_scores.best_estimator_

LGBMClassifier(bagging_fraction=0.8, bagging_freq=0, class_weight='balanced',
               colsample_bytree=0.4, feature_fraction=0.8, max_depth=4,
               min_child_samples=10, n_estimators=30, num_leaves=8,
               reg_alpha=0.2, subsample_for_bin=500)

In [ ]:
lgb_scores.best_score_


In [254]:
lgb_es = lgb_scores.best_estimator_.fit(nX, y)
lgb_es.predict(test_X)

array(['G1', 'G4', 'G4', 'G2', 'G4', 'G4', 'G4', 'G4'], dtype='<U2')

In [ ]:
params = {"min_weight_fraction_leaf": np.arange(0.01, 0.4, 0.1)}

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [i for i in range(1, 15, 2)], 
    "max_features": ("sqrt", "log2"),
    'criterion': ["gini", "entropy"],
    "min_samples_split": [i for i in range(2, 8, 1)], 
    
}

dt_scores = GS_training(nX, y, params, DecisionTreeClassifier, cv=7)

In [247]:
dt_scores.best_score_ # 0.6266666666666667

0.7346938775510204

In [250]:
dt_es = dt_scores.best_estimator_.fit(nX, y)
dt_es.predict(test_X)

array(['G4', 'G4', 'G4', 'G2', 'G4', 'G4', 'G4', 'G2'], dtype='<U2')

In [ ]:
params = {"min_samples_leaf": [i for i in range(2, 16, 4)], 
          "min_samples_split": [i for i in range(2, 5, 1)], 
          "max_features": ("auto", "sqrt", "log2"),
         "min_weight_fraction_leaf": np.arange(0.05, 0.5, 0.1)}
params = {'n_estimators': [5, 20, 50, 100], 
 'max_features': ['auto', 'sqrt'], 
 'max_depth': [10, 20, 30, 40], 
 'min_samples_split': [2, 6, 10], 
 'min_samples_leaf': [1, 3, 4], 
 'bootstrap': [True, False]}
rf_scores = GS_training(nX, y, params, RandomForestClassifier, cv=7)

In [238]:
rf_scores.best_score_ # 0.7

0.7959183673469388

In [241]:
rf_es = rf_scores.best_estimator_.fit(nX, y)
rf_es.predict(test_X)

array(['G1', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G2'], dtype='<U2')

In [ ]:
params = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
    'algorithm': ['SAMME', 'SAMME.R']
}

ada_scores = GS_training(nX, y, params, AdaBoostClassifier)

In [249]:
ada_scores.best_score_

0.6955555555555556

In [251]:
ada_es = ada_scores.best_estimator_.fit(nX, y)
ada_es.predict(test_X)

array(['G1', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4'], dtype='<U2')